In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
import seaborn as sns

import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go 

url = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/spotify.zip"
df_music = pd.read_csv(url)

df_music2 = df_music.copy()
df_music2.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [2]:
df_music2.drop(columns=['time_signature', 'key', 'track_id',], inplace=True)

In [3]:
df_music2[(df_music2['genre'] == 'Children’s Music') | (df_music2['genre'] == "Children's Music")]


,genre,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,valence
71649,Children's Music,Toddler Tunes,Itsy Bitsy Spider,47,0.302000,0.901,137454,0.526,0.000000,0.0540,-7.296,Minor,0.0362,109.946,0.706
71650,Children's Music,Spongebob Squarepants,The Best Day Ever,47,0.202000,0.565,182173,0.758,0.000000,0.0552,-5.205,Major,0.0294,127.817,0.605
71651,Children's Music,Blippi,The Excavator Song,46,0.528000,0.737,175187,0.549,0.000087,0.2990,-13.215,Major,0.0382,124.980,0.840
71652,Children's Music,Pinkfong,Baby Shark Music Box,44,0.688000,0.686,110523,0.121,0.885000,0.0793,-20.751,Major,0.1030,148.002,0.818
71653,Children's Music,Toddler Tunes,"Head, Shoulders, Knees and Toes",47,0.249000,0.867,83291,0.729,0.000000,0.0802,-6.616,Major,0.0373,120.129,0.793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104017,Children’s Music,Smash Mouth,Then The Morning Comes,46,0.024000,0.736,182267,0.868,0.000000,0.0735,-4.972,Minor,0.0289,118.599,0.961
104018,Children’s Music,Built To Spill,Liar,52,0.266000,0.537,311347,0.755,0.090100,0.2620,-8.187,Major,0.0269,134.879,0.538
104019,Children’s Music,Moon Taxi,Let The Record Play,44,0.004400,0.672,188667,0.855,0.160000,0.1670,-4.293,Minor,0.0785,166.025,0.779
104020,Children’s Music,Rend Collective,Every Giant Will Fall,49,0.019800,0.551,225320,0.847,0.000000,0.1320,-6.909,Major,0.0456,111.975,0.445


In [4]:
df_music2.drop(df_music2[(df_music2['genre'] == 'Soundtrack') | (df_music2['genre'] == 'Movie')].index, inplace=True)
df_music2 = df_music2.reset_index(drop=True)

In [5]:
df_music2['genre'].replace("Children’s Music", "Children's Music", inplace=True)


In [6]:
(df_music2['genre'] == "Children's Music").value_counts()

False    200517
True      14756
Name: genre, dtype: int64

In [7]:
#transform ms en s
df_music2['duration_ms'] = round(df_music2['duration_ms']/1000,2)

In [8]:
#rename columns ms
df_music2.rename(columns={'duration_ms':'duration_sec'},inplace=True)


In [9]:
popularity_genre = round(df_music2.groupby(by=['genre']).mean('popularity'),2).sort_values(['popularity'], ascending=False).reset_index()


In [10]:
fig = px.scatter_polar(popularity_genre, r="popularity", theta="genre",
                       color="popularity", size="popularity",
                       color_continuous_scale=px.colors.diverging.Tealrose,
                        
                       template="plotly_dark")

fig.update_layout(autosize=False,
                    title='Popularity rate for each musical genre',
                    title_x = 0.1,                    
                    width=1000,
                    height=600,                    
                    template='plotly_dark',
                    font_size=13
                    )

fig.show()

In [11]:

fig = px.line_polar(popularity_genre, r="danceability", theta="genre", line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",
                    )

fig.update_traces(fill='toself')

fig.update_layout(autosize=False, 
                    title='Danceability for each musical genre',
                    title_x = 0.1,                    
                    width=1000,
                    height=600,                    
                    template='plotly_dark',
                    font_size=13
                    )

fig.show()